In [154]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [155]:
EPOCHS = 2
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '/home/saurav/Desktop/sih(saurav)/sikkimgov/dataset/'
width=256
height=256
depth=3

In [156]:
# Function to convert images to array


def convert_image_to_array(image_dir):
#     try:
    image = cv2.imread(image_dir)
    if image is not None :
        image = cv2.resize(image, default_image_size)  
        return img_to_array(image)
#     else :
#         return np.array([])
#     except Exception as e:
#         print(f"Error : {e}")
#         return None


In [157]:
# Fetch images from directory
image_list, label_list = [], []

print("[INFO] Loading images ...")
root_dir = listdir(directory_root)



for train_folder in root_dir :
    total_folder_list = listdir(f"{directory_root}/{train_folder}")
    
    for phase_folder in total_folder_list:
        print(f"[INFO] Processing {phase_folder} ...")
        phase_image_list = listdir(f"{directory_root}/{train_folder}/{phase_folder}/")
#         print(phase_image_list)
                
        for image in phase_image_list[:50] :
            image_directory = f"{directory_root}/{train_folder}/{phase_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True or image_directory.endswith(".jpeg") == True or image_directory.endswith(".JPEG") == True:
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(phase_folder)
            
                
print("Image loading completed")  

[INFO] Loading images ...
[INFO] Processing PHASE_1 ...
[INFO] Processing PHASE_6 ...
[INFO] Processing PHASE_3 ...
[INFO] Processing PHASE_2 ...
[INFO] Processing PHASE_5 ...
[INFO] Processing PHASE_4 ...
Image loading completed


In [158]:
image_size = len(image_list)
print(image_size)

298


In [159]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

print(label_binarizer.classes_)


['PHASE_1' 'PHASE_2' 'PHASE_3' 'PHASE_4' 'PHASE_5' 'PHASE_6']


In [160]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0


In [161]:
print("Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 
print(len(x_train))

Spliting data to train, test
238


In [162]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")


In [163]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1

In [164]:
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [165]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
activation_22 (Activation)   (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_23 (Activation)   (None, 85, 85, 64)       

In [166]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("training network...")

training network...


In [167]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=12,
    #len(x_train) // BS
    epochs=EPOCHS, verbose=1
    )

Epoch 1/2
12/12 [==============================] - 276s 23s/step - loss: 0.6477 - accuracy: 0.7992 - val_loss: 0.5759 - val_accuracy: 0.7389
Epoch 2/2
12/12 [==============================] - 59s 5s/step - loss: 0.4654 - accuracy: 0.8396 - val_loss: 0.6657 - val_accuracy: 0.7389


In [168]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
# print(history.history)

In [169]:
model._make_predict_function()
# Train and validation accuracy
# plt.plot(epochs, acc, 'b', label='Training accurarcy')
# plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
# plt.title('Training and Validation accurarcy')
# plt.legend()

# plt.figure()
# # Train and validation loss
# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and Validation loss')
# plt.legend()
# plt.show()


# Model Accuracy

In [170]:
print("model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Accuracy: {scores[1]*100}")


model accuracy
60/60 [==============================] - 2s 28ms/step
Accuracy: 73.88889193534851


In [171]:
# Save model using Pickle

print("Saving model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))


Saving model...


In [172]:
from PIL import Image
final_test = convert_image_to_array('/home/saurav/Desktop/sih(saurav)/sikkimgov/single_prediction/image9.jpg')

final_test_ = np.array(final_test, dtype=np.float16) / 225.0
final_test_resized = np.resize(final_test_,(1,256,256,3))
xi = model.predict_classes(final_test_resized,batch_size=BS,verbose=0)
img_4=array_to_img(final_test)
img_4.save('img_4.jpg')
print(label_binarizer.classes_[xi])
print(xi)
# img = Image.fromarray(final_test_, 'RGB')

# img.show()




['PHASE_2']
[1]
